In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import sys
sys.path.append('../segmention_buildings/BuildFormer')
import matplotlib.pyplot as plt
import json
import sys
from torch  import nn
from tqdm import tqdm
from torch import optim
import torch.nn.functional as F
import cv2
import numpy as np
import kornia as K
import math
from glob import glob
from os.path import join
import torch
from easydl import clear_output

[easydl] tensorflow not available!


In [14]:
from init import *
from train_model import train_model
from lossers import dice_loss
from  models import get_model
from lossers import get_loss_function
from datasets import get_dataloader
from geoseg.models.BuildFormer import BuildFormerSegDP
from optimizers_schedulers import get_optimizer_scheduler, update_paramter
from utiles.mask_utles import get_color_mask,mask2color,show_image_mask,show_image_mask_, rgbmask2label,label_colormap, image_diff
from utiles.mask_utles import rgbmask2label,label2rgbmask,rgb2cls_masks,label2rgbmask2,label_colormap
from utiles.tensor2imgmask import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import sys
sys.path.append('/home/wanghaifeng/whf_work/work_sync/vit/vit-pytorch/vit_pytorch')
sys.path.append('/home/wanghaifeng/whf_work/work_sync/satellite_data/models_config/segment-anything/segment_anything')
# from  vit  import  ViT
# from image_encoder import *

In [16]:
def vis_result(results,result_vis_path):
    data_dict= {"TP":[], "TN":[], "FP":[], "FN":[], "PPV":[], "TPR":[], "TPR":[], "TNR":[], "F1":[], "Dice":[], "IOU":[],"model":[],"data":[],"times":[]}
    flag = time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime())
    for i,d in enumerate(results):
        keys =list(data_dict.keys())
        for d0,k in zip(d[0],keys[:len(d[0])]):
            data_dict[k].append(d0)
        for d1,k in zip(d[1],keys[len(d[0]):len(d[0])+len(d[1])]):
            data_dict[k].append(d1)
        
        # data_dict["times"].append(i)
        # data_dict["id"].append(i)
        
    data_t = pandas.DataFrame(data_dict,columns = keys)
    data_t.to_html(join(result_vis_path,flag+'.html'))
    data_t.to_html(join(result_vis_path,'result.html'))
    data_t.to_json(join(result_vis_path,flag+'.json'))
    data_t.to_csv(join(result_vis_path,flag+'.csv'))

In [ ]:
from metrics.evel_models  import  evel_models,get_model_dataset
from glob import glob
import pandas
import time
from os.path import join

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_roots =['/home/wanghaifeng/project_work/datasets/disaster_dataset/harvey']
save_root = '/home/wanghaifeng/whf_work/model_works/model_vis_result'
modes_config  = glob(r'/home/wanghaifeng/whf_work/work_sync/satellite_data/models_config/cfdnet/model_works/vgg_unet/vgg_unet_2024_01_31_20_48_13/model_config.json')

results = evel_models(save_root =save_root,modes_config=modes_config,dataset_roots=dataset_roots,device=device,
            models_predection=True,models_metrics=False,img_size = (1024,1024),cross_dataset_test = False,min_num_limt=1,
                     show_img=True,show_mask=True,show_gtmaskimg=True,show_premaskimg=True,show_fixedmaskimg=True,save_mask=True,save_fixedmask=True)
# clear_output()
# result_vis_path = '/home/wanghaifeng/whf_work/work_sync/satellite_data/valite_dataset_sys/vis'
# vis_result(results,result_vis_path)